<a href="https://colab.research.google.com/github/rytaylor/FairFace/blob/resnext/ff_resnet_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start by mounting the drive location to get the CSVs & unzip the training data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Then read these locations into memory, extracting the zip to a temporary local location to be read as needed:

In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
import imageio as io

img_zip_path = '/content/drive/My Drive/FairFace-Colab/fairface-img-margin025-trainval.zip'
zip_ref = zipfile.ZipFile(img_zip_path, 'r')
zip_ref.extractall('/tmp/model_data')
zip_ref.close()

#print(io.imread('/fairface-img-margin025-trainval/train/1.jpg'))

train_data_path = '/content/drive/My Drive/FairFace-Colab/fairface_label_train.csv'
train_data_df = pd.read_csv(train_data_path)

val_data_path = '/content/drive/My Drive/FairFace-Colab/fairface_label_val.csv'
val_data_df = pd.read_csv(train_data_path)

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True


Now the dataset class is defined, which allows the model to grab the training & validation data and implements the label tensor logic:

In [ ]:
import torchvision
from glob import glob
import os
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from torch.utils.data import Dataset
%matplotlib inline

class FaceImageDataset(torch.utils.data.Dataset):

    def __init__(self, csv_path, rootdir, transform=None):
        self.csv_file = pd.read_csv(csv_path)
        self.rootdir = rootdir
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        img_name = os.path.join(self.rootdir, self.csv_file.iloc[index, 0])
        image = io.imread(img_name)

        age = self.csv_file.iloc[index, 1]
        gender = self.csv_file.iloc[index, 2]
        race = self.csv_file.iloc[index, 3]
        service_test = self.csv_file.iloc[index, 4]

        if(self.transform):
            image = self.transform(image)

        age_list = ['0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', 'more than 70']
        race_list = ['White', 'Black', 'Latino_Hispanic', 'East Asian', 'Southeast Asian', 'Indian', 'Middle Eastern']
        gender_list = ['Male', 'Female']

        service_test = 1 if service_test == True else 0

        label = torch.zeros([18])
        label[age_list.index(age)] = 1
        label[9+race_list.index(race)] = 1
        label[16+gender_list.index(gender)] = 1

        label = torch.as_tensor(label, dtype=torch.float64)

        sample = {'image': image, 'label': label}

        return sample

Two datasets are defined, training and validation, with a set of transforms to match what a pretrained ResNet expects:

In [ ]:
data_transforms = torchvision.transforms.Compose([transforms.ToPILImage(),
                                                  transforms.Resize((224, 224)),
                                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                                  transforms.ToTensor()])
train_dl = FaceImageDataset(csv_path=train_data_path, rootdir='/tmp/model_data/', transform=data_transforms)
valid_dl = FaceImageDataset(csv_path=val_data_path, rootdir='/tmp/model_data/', transform=data_transforms)

DataLoaders are created to wrap the above dataset classes for use by the model in training:

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = False, batch_size = 64, num_workers = 3)
val_dataloader = torch.utils.data.DataLoader(valid_dl, shuffle = True, batch_size = 64, num_workers = 3)

Here, we define the model; it is a ResNet50 with adapted fully-connected layers and sigmoid reshaping of the final linear layer.

In [ ]:
class ResnetModel(nn.Module):
  def __init__(self):
    super(ResnetModel, self).__init__()
    self.model = torchvision.models.resnet50(pretrained=True)
    self.model.fc = nn.Sequential(nn.Linear(2048, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 18))
    self.sigmoid = nn.Sigmoid()
    #
    self.cuda()
    #self.model.cuda()
  def forward(self, x):
    x = self.model(x)
    return self.sigmoid(x)

The below function is a helper to get the accuracy of a given prediction relative to the original data

In [ ]:
def pred_acc(original, predicted):
  return torch.round(predicted).eq(original).sum().numpy()/len(original) 

Now, the training loop is defined with binary cross-entropy loss for multilabel classifying as well as a stochastic gradient descent optimizer with default parameters; this is pretty much boilerplate model fitting for pytorch. 

In [ ]:
from pprint import pprint

def fit_model(epochs, model, dataloader, phase = 'training', volatile = False):

    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-5)
    
    pprint("Epoch: {}".format(epochs))
    if phase == 'training':
        model.train()
        
    if phase == 'validation':
        model.eval()
        volatile = True
        
    running_loss = []
    running_accuracy = []
    for i, data in enumerate(dataloader):
        
        inputs, target = data['image'].cuda(), data['label'].float().cuda()
        
        inputs, target = Variable(inputs), Variable(target)
        
        if phase == 'training':
            optimizer.zero_grad()
            
        ops = model(inputs)
        accuracy_out = []

        for i, d in enumerate(ops, 0):
          accuracy = pred_acc(torch.Tensor.cpu(target[i]), torch.Tensor.cpu(d))
          accuracy_out.append(accuracy)
          
        loss = criterion(ops, target)
                        
        running_loss.append(loss.item())
        running_accuracy.append(np.asarray(accuracy_out).mean())
        
        if phase == 'training':
            
            loss.backward()
            optimizer.step()
              
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_accuracy = np.asarray(running_accuracy).mean()
    
    pprint("{} loss is {} ".format(phase,total_batch_loss))
    pprint("{} accuracy is {} ".format(phase, total_batch_accuracy))
    
    return total_batch_loss, total_batch_accuracy

def check_cuda():
    if torch.cuda.is_available():
        return True
    return False

is_cuda = check_cuda()
model = ResnetModel()
if is_cuda:
    model.cuda()

And below is the call to fit_model for the training and validation items.

In [ ]:
from tqdm import tqdm
import imageio as io
import time

train_loss = []
train_accuracy = []
val_loss = []
val_accuracy = []
for i in tqdm(range(1, 6)):
  train_l, train_a = fit_model(i, model, train_dataloader)
  val_l, val_a = fit_model(i, model, val_dataloader, phase = 'validation')
  train_loss.append(trn_l); train_accuracy.append(trn_a)
  val_loss.append(val_l); val_accuracy.append(val_a)
  torch.save(model, "drive/My Drive/FairFace-Colab/model_res_{}".format(i))
torch.save(model, "drive/My Drive/FairFace-Colab/model_res_final")






  0%|          | 0/4 [00:00<?, ?it/s]

'Epoch: 1'
'training loss is 0.3200726832841341 '
'training accuracy is 0.8680049643428008 '
'Epoch: 1'







 25%|██▌       | 1/4 [06:09<18:29, 369.85s/it]

'validation loss is 0.30736675279853987 '
'validation accuracy is 0.8726591646390915 '
'Epoch: 2'
'training loss is 0.2943079737401545 '
'training accuracy is 0.8771498063445223 '
'Epoch: 2'







 50%|█████     | 2/4 [12:11<12:14, 367.24s/it]

'validation loss is 0.2904757005237315 '
'validation accuracy is 0.8788381995133819 '
'Epoch: 3'
'training loss is 0.2742673765163015 '
'training accuracy is 0.8844848917988444 '
'Epoch: 3'







 75%|███████▌  | 3/4 [18:14<06:06, 366.21s/it]

'validation loss is 0.2818192746734967 '
'validation accuracy is 0.8822566909975669 '
'Epoch: 4'
'training loss is 0.258962304026647 '
'training accuracy is 0.8902913541538588 '
'Epoch: 4'







100%|██████████| 4/4 [24:19<00:00, 364.84s/it]

'validation loss is 0.28163704589335586 '
'validation accuracy is 0.8829643146796431 '


Here is an alternate form allowing for retraining of a model from a saved compressed model file.

In [ ]:
model = MultiClassifier()
model = torch.load('drive/My Drive/FairFace-Colab/model6')
from tqdm import tqdm
import imageio as io
import time

if is_cuda:
  model.cuda()
trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(1, 11)):
  trn_l, trn_a = fit_model(i, model, train_dataloader)
  val_l, val_a = fit_model(i, model, val_dataloader, phase = 'validation')
  trn_losses.append(trn_l); trn_acc.append(trn_a)
  val_losses.append(val_l); val_acc.append(val_a)
torch.save(model, "drive/My Drive/FairFace-Colab/model7")

NameError: ignored

==



Below is the full training implementation to run all at once:



==

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torchvision
from glob import glob
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from torch.utils.data import Dataset
%matplotlib inline
import zipfile
import pandas as pd
import numpy as np
import imageio as io


from google.colab import drive
drive.mount('/content/drive')

rootd = '/content/drive/My Drive/FairFace-Colab/'

img_zip_path = os.path.join(rootd,'fairface-img-margin025-trainval.zip')
zip_ref = zipfile.ZipFile(img_zip_path, 'r')
zip_ref.extractall('/tmp/model_data')
zip_ref.close()

train_data_path = os.path.join(rootd,'fairface_label_train.csv')
train_data_df = pd.read_csv(train_data_path)

val_data_path = os.path.join(rootd,'fairface_label_val.csv')
val_data_df = pd.read_csv(train_data_path)
val_data_df.head()

class FaceImageDataset(torch.utils.data.Dataset):

    def __init__(self, csv_path, rootdir, transform=None):
        self.csv_file = pd.read_csv(csv_path)
        self.rootdir = rootdir
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        img_name = os.path.join(self.rootdir, self.csv_file.iloc[index, 0])
        image = io.imread(img_name)

        age = self.csv_file.iloc[index, 1]
        gender = self.csv_file.iloc[index, 2]
        race = self.csv_file.iloc[index, 3]
        service_test = self.csv_file.iloc[index, 4]

        if(self.transform):
            image = self.transform(image)

        age_list = ['0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', 'more than 70']
        race_list = ['White', 'Black', 'Latino_Hispanic', 'East Asian', 'Southeast Asian', 'Indian', 'Middle Eastern']
        gender_list = ['Male', 'Female']

        service_test = 1 if service_test == True else 0

        label = torch.zeros([18])
        label[age_list.index(age)] = 1
        label[9+race_list.index(race)] = 1
        label[16+gender_list.index(gender)] = 1

        label = torch.as_tensor(label, dtype=torch.float64)

        sample = {'image': image, 'label': label}

        return sample

data_transforms = torchvision.transforms.Compose([transforms.ToPILImage(),
                                                  transforms.Resize((224, 224)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                  ])
train_dl = FaceImageDataset(csv_path=train_data_path, rootdir='/tmp/model_data/', transform=data_transforms)
valid_dl = FaceImageDataset(csv_path=val_data_path, rootdir='/tmp/model_data/', transform=data_transforms)

#print(train_dl.__getitem__(0))

train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = False, batch_size = 16, num_workers = 3)
val_dataloader = torch.utils.data.DataLoader(valid_dl, shuffle = True, batch_size = 16, num_workers = 3)

class ResnetModel(nn.Module):
  def __init__(self):
    super(ResnetModel, self).__init__()
    self.model = torchvision.models.resnet50(pretrained=True)
    self.model.fc = nn.Sequential(nn.Linear(2048, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 18))
    self.sigmoid = nn.Sigmoid()
    #
    self.cuda()
    #self.model.cuda()
  def forward(self, x):
    x = self.model(x)
    return self.sigmoid(x)

def pred_acc(original, predicted):
  return torch.round(predicted).eq(original).sum().numpy()/len(original) 

from pprint import pprint

def fit_model(epochs, model, dataloader, phase = 'training', volatile = False):

    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-5)
    
    pprint("Epoch: {}".format(epochs))
    if phase == 'training':
        model.train()
        
    if phase == 'validataion':
        model.eval()
        volatile = True
        
    running_loss = []
    running_acc = []
    b = 0
    for i, data in enumerate(dataloader):
        
        inputs, target = data['image'].cuda(), data['label'].float().cuda()
        #add .cuda() to above
        
        inputs, target = Variable(inputs), Variable(target)
        
        if phase == 'training':
            optimizer.zero_grad()
            
        ops = model(inputs)
        acc_ = []

        for i, d in enumerate(ops, 0):
          acc = pred_acc(torch.Tensor.cpu(target[i]), torch.Tensor.cpu(d))
          acc_.append(acc)
          
        loss = criterion(ops, target)
                        
        running_loss.append(loss.item())
        running_acc.append(np.asarray(acc_).mean())
        b += 1
        
        if phase == 'training':
            
            loss.backward()
            optimizer.step()
              
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_acc = np.asarray(running_acc).mean()
    
    pprint("{} loss is {} ".format(phase,total_batch_loss))
    pprint("{} accuracy is {} ".format(phase, total_batch_acc))
    
    return total_batch_loss, total_batch_acc

def check_cuda():
    _cuda = False
    if torch.cuda.is_available():
        _cuda = True
    return _cuda

is_cuda = check_cuda()

model = ResnetModel()
###
model = torch.load(os.path.join(rootd,'model_res2_2'))

#if is_cuda:
#    model.cuda()

from tqdm import tqdm
import imageio as io
import time


trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(1, 31)):
  trn_l, trn_a = fit_model(i, model, train_dataloader)
  val_l, val_a = fit_model(i, model, val_dataloader, phase = 'validation')
  trn_losses.append(trn_l); trn_acc.append(trn_a)
  val_losses.append(val_l); val_acc.append(val_a)
  torch.save(model, os.path.join(rootd, 'model_res3_{}'.format(i)))
torch.save(model, os.path.join(rootd, 'model_res3_final'))

Mounted at /content/drive


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0/30 [00:00<?, ?it/s]

'Epoch: 1'
'training loss is 0.1212176025409852 '
'training accuracy is 0.9483092288618385 '
'Epoch: 1'
'validation loss is 0.2722037834733942 '
'validation accuracy is 0.8964517437145174 '


  3%|▎         | 1/30 [12:43<6:08:56, 763.33s/it]

...

 93%|█████████▎| 28/30 [5:57:52<25:41, 770.93s/it]

'Epoch: 29'
'training loss is 0.014216043666828415 '
'training accuracy is 0.9950599922127955 '
'Epoch: 29'
'validation loss is 0.5294592761645351 '
'validation accuracy is 0.8972810218978101 '


 97%|█████████▋| 29/30 [6:10:51<12:53, 773.20s/it]

'Epoch: 30'
'training loss is 0.013827420693724684 '
'training accuracy is 0.9951515687118323 '
'Epoch: 30'
'validation loss is 0.5362720081188382 '
'validation accuracy is 0.897875101378751 '


100%|██████████| 30/30 [6:23:42<00:00, 767.41s/it]


Below is the implementation for predicting the classification for an input image, all at once.


*   *detect_face* is an identical implementation from the FairFace predict.py script.
*   *predict* is mostly standard pytorch usage, formatted to better display results for each label.

*   *get_tensor* implements the same transforms as above.



In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torchvision
from glob import glob
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from torch.utils.data import Dataset
%matplotlib inline
import zipfile
import pandas as pd
import numpy as np
import imageio as io
import dlib

rootd = '/content/drive/My Drive/FairFace-Colab/'

class ResnetModel(nn.Module):
  def __init__(self):
    super(ResnetModel, self).__init__()
    self.model = torchvision.models.resnet50(pretrained=True)
    self.model.fc = nn.Sequential(nn.Linear(2048, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 18))
    self.sigmoid = nn.Sigmoid()
    #
    self.cuda()
    #self.model.cuda()
  def forward(self, x):
    x = self.model(x)
    return self.sigmoid(x)

def predict(img, label_lst, model):
  img = detect_face(img)
  tnsr = get_tensor(img)
  op = model(tnsr)
  op_b = torch.round(op)
  op_b_np = torch.Tensor.cpu(op_b).detach().numpy()
  preds = np.where(op_b_np == 1)[1]
  sigs_op = torch.Tensor.cpu(torch.round((op)*100)).detach().numpy()[0]
  o_p = np.argsort(torch.Tensor.cpu(op).detach().numpy())[0][::-1]
  age = label_lst[0][np.argmax(sigs_op[:9])]
  race = label_lst[1][np.argmax(sigs_op[9:16])]
  gender = label_lst[2][np.argmax(sigs_op[16:])]
  print(sigs_op)
  return [[race, np.max(sigs_op[9:16])], [gender, np.max(sigs_op[16:])], [age, np.max(sigs_op[:9])]]

def detect_face(image_path, default_max_size=800,size = 300, padding = 0.25):
    cnn_face_detector = dlib.cnn_face_detection_model_v1(os.path.join(rootd, 'dlib_models/mmod_human_face_detector.dat'))
    sp = dlib.shape_predictor(os.path.join(rootd, 'dlib_models/shape_predictor_5_face_landmarks.dat'))
    base = 2000  # largest width and height
    img = dlib.load_rgb_image(image_path)

    old_height, old_width, _ = img.shape

    if old_width > old_height:
        new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
    else:
        new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
    img = dlib.resize_image(img, rows=new_height, cols=new_width)

    dets = cnn_face_detector(img, 1)
    num_faces = len(dets)
    if num_faces == 0:
        print("Sorry, there were no faces found in '{}'".format(image_path))
        return
    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in dets:
        rect = detection.rect
        faces.append(sp(img, rect))
    image = dlib.get_face_chips(img, faces, size=size, padding = padding)
    return image[0]

labels = [['0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', 'more than 70'],
          ['White', 'Black', 'Latino_Hispanic', 'East Asian', 'Southeast Asian', 'Indian', 'Middle Eastern'],
          ['Male', 'Female']]

import imageio as io
import numpy as np

def get_tensor(img):
  tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  return tfms(np.asarray(img)).unsqueeze(0)

model_path = os.path.join(rootd, 'model_res3_final')
model = torch.load(model_path, map_location=torch.device('cpu'))
model = model.eval()
predict(os.path.join(rootd, 'test/squid.jpg'), labels, model)

[ 0.  0.  0. 75. 23.  1.  3.  0.  0.  0.  0.  0. 99.  0.  0.  0. 96.  4.]


[['East Asian', 99.0], ['Male', 96.0], ['20-29', 75.0]]

The model was also manually tested on the original set of test_imgs from FairFace, as well as manually inserted images from the Internet. The final model used was trained for 45 epochs, selected for from two previous models. The validation loss (and accuracy) stabilized quickly, but there was volatility in many of the race classifications throughout much of the training when comparing each successive generation. The most firm predictions came from this final model version, with relatively equal performance per individual classification to the human eye. 

In [ ]:
import torch.onnx
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torchvision
from glob import glob
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from torch.utils.data import Dataset
%matplotlib inline
import zipfile
import pandas as pd
import numpy as np
import imageio as io
import dlib

class ResnetModel(nn.Module):
  def __init__(self):
    super(ResnetModel, self).__init__()
    self.model = torchvision.models.resnet50(pretrained=True)
    self.model.fc = nn.Sequential(nn.Linear(2048, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 18))
    self.sigmoid = nn.Sigmoid()
    #
    self.cuda()
    #self.model.cuda()
  def forward(self, x):
    x = self.model(x)
    return self.sigmoid(x)


rootd = '/content/drive/My Drive/FairFace-Colab/'
device = torch.device('cpu')
model = torch.load(os.path.join(rootd, 'model_res3_final'), map_location=device)
dummy_input = torch.randn(1, 3, 224, 224)
model.eval()
# trace model with a dummy input
traced_model = torch.jit.trace(model, dummy_input)
traced_model.save((os.path.join(rootd, 'res3f_traced.pt')))


In [ ]:
!pip install pipreqs
!pip install nbconvert

In [ ]:
!cd '/content/drive/My Drive/ffrn'
!pipreqs

ERROR: Failed on file: /content/drive/MyDrive/ffrn/ff_resnet.py
Traceback (most recent call last):
  File "/usr/local/bin/pipreqs", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 488, in main
    init(args)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 418, in init
    follow_links=follow_links)
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 131, in get_all_imports
    raise exc
  File "/usr/local/lib/python3.7/dist-packages/pipreqs/pipreqs.py", line 117, in get_all_imports
    tree = ast.parse(contents)
  File "/usr/lib/python3.7/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 615
    !pip install pipreqs
    ^
SyntaxError: invalid syntax
